In [40]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

In [41]:
EPS = 11
MIN_POINTS = 5

fake_data = pd.read_csv("dupremoved_trackdata.csv", sep = ",")

# Column 9, "mode", has been removed to increase sanity of results, as it restricted DBScan to only 2 possible clusters
# put as '8' back into the hardcoded array of numbers to add it back in
# Column 10, time was milliseconds & has been removed for obvious reasons
data = fake_data.to_numpy()[:, [4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 17]].astype(np.double)

scanner = DBSCAN(eps=EPS, min_samples=MIN_POINTS).fit(data)
labels = scanner.labels_
bi = [bool(i) for i in (labels + 1)]
bool_idx = np.array(bi)

cleaned_data = fake_data.to_numpy()[bool_idx]
cleaned_dataframe = pd.DataFrame(data=cleaned_data, columns=fake_data.columns, index=fake_data.index[bool_idx])
cleaned_dataframe.to_csv("dbscaned_trackdata.csv", sep=",")

In [42]:
print("Number of clusters: ", len(set(labels)) - (1 if -1 in labels else 0))
if (len(cleaned_data)):
    print("# of removed songs: ", np.shape(data)[0] - np.shape(cleaned_data)[0])
else:
    print("Removed all songs")

Number of clusters:  1
# of removed songs:  4
